<a href="https://colab.research.google.com/github/Sansustito/ExamenPython/blob/master/Forecasting_con_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOTA IMPORTANTE

Para utilizar este Notebook correctamente se debe tener en cuenta que Prophet requiere dos columnas de datos, la primera deben ser fechas y la segunda los datos históricos, mínimo de dos años (base diaria). Prophet puede trabajar con menos datos pero no captura correctamente estacionalidades.


In [ ]:
# CELDA 1: Instalación de librerías
!pip install prophet plotly ipywidgets kaleido openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.6 MB/s eta 0:00:00


In [ ]:
# CELDA 2: Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
from datetime import datetime, timedelta
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, HTML
import io

# Configuraciones
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [ ]:
# CELDA 3: Widget para carga de datos
def create_upload_widget():
    upload_widget = widgets.FileUpload(
        accept='.csv',
        multiple=False,
        description='Seleccionar CSV'
    )

    output_widget = widgets.Output()

    def on_upload_change(change):
        with output_widget:
            output_widget.clear_output()
            if change['new']:
                uploaded_file = list(change['new'].values())[0]
                content = uploaded_file['content']
                df_string = content.decode('utf-8')

                global df_raw
                df_raw = pd.read_csv(io.StringIO(df_string))

                print(f"✅ Archivo cargado: {uploaded_file['metadata']['name']}")
                print(f"📊 Dimensiones: {df_raw.shape}")
                print(f"📅 Columnas: {list(df_raw.columns)}")
                print("\n🔍 Primeras 5 filas:")
                print(df_raw.head())
                print(f"\n📈 Estadísticas básicas:")
                print(df_raw.describe())

    upload_widget.observe(on_upload_change, names='value')

    print("📁 Widget de carga creado")
    print("👆 Usa el widget para subir tu archivo CSV")

    return widgets.VBox([
        widgets.HTML("<h3>📁 Carga de Datos</h3>"),
        upload_widget,
        output_widget
    ])

# Mostrar widget
upload_interface = create_upload_widget()
display(upload_interface)

📁 Widget de carga creado
👆 Usa el widget para subir tu archivo CSV


In [ ]:
# CELDA 4: Preprocesamiento de datos
def preprocess_data(df):
    # Crear copia del DataFrame
    df_processed = df.copy()

    # Convertir fecha a datetime
    df_processed['fecha'] = pd.to_datetime(df_processed['fecha'])

    # Renombrar columnas para Prophet
    df_processed = df_processed.rename(columns={
        'fecha': 'ds',
        'unidades_vendidas': 'y'
    })

    # Ordenar por fecha
    df_processed = df_processed.sort_values('ds').reset_index(drop=True)

    # Verificar datos faltantes
    missing_dates = pd.date_range(
        start=df_processed['ds'].min(),
        end=df_processed['ds'].max()
    ).difference(df_processed['ds'])

    if len(missing_dates) > 0:
        print(f"⚠️ Fechas faltantes encontradas: {len(missing_dates)}")
        print("🔧 Completando fechas faltantes con interpolación...")

        # Crear rango completo de fechas
        full_date_range = pd.date_range(
            start=df_processed['ds'].min(),
            end=df_processed['ds'].max()
        )

        # Crear DataFrame con todas las fechas
        full_df = pd.DataFrame({'ds': full_date_range})

        # Merge y llenar valores faltantes
        df_processed = full_df.merge(df_processed, on='ds', how='left')
        df_processed['y'] = df_processed['y'].interpolate(method='linear')

    print("✅ Datos procesados correctamente")
    print(f"📊 Período: {df_processed['ds'].min()} a {df_processed['ds'].max()}")
    print(f"📈 Total de observaciones: {len(df_processed)}")
    print(f"🎯 Promedio diario: {df_processed['y'].mean():.1f} unidades")
    print(f"📊 Desviación estándar: {df_processed['y'].std():.1f}")

    return df_processed

# Ejecutar preprocesamiento
try:
    if 'df_raw' in globals():
        df_prophet = preprocess_data(df_raw)
        print("\n🎉 Dataset listo para Prophet!")
    else:
        print("⚠️ Primero carga el archivo CSV")
except NameError:
    print("⚠️ Primero carga el archivo CSV")

✅ Datos procesados correctamente
📊 Período: 2022-01-01 00:00:00 a 2024-12-31 00:00:00
📈 Total de observaciones: 1096
🎯 Promedio diario: 154.1 unidades
📊 Desviación estándar: 84.3

🎉 Dataset listo para Prophet!


In [ ]:
# CELDA 5: Análisis exploratorio de datos
def exploratory_analysis(df):
    # Configurar el subplot
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Serie Temporal Completa', 'Distribución de Ventas',
                       'Tendencia por Mes', 'Patrones Semanales'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Gráfico 1: Serie temporal completa
    fig.add_trace(
        go.Scatter(
            x=df['ds'],
            y=df['y'],
            mode='lines',
            name='Unidades Vendidas',
            line=dict(color='#1f77b4', width=1)
        ),
        row=1, col=1
    )

    # Gráfico 2: Histograma de distribución
    fig.add_trace(
        go.Histogram(
            x=df['y'],
            nbinsx=30,
            name='Distribución',
            marker_color='#ff7f0e',
            opacity=0.7
        ),
        row=1, col=2
    )

    # Preparar datos para análisis mensual
    df_monthly = df.copy()
    df_monthly['month'] = df_monthly['ds'].dt.month
    monthly_avg = df_monthly.groupby('month')['y'].mean().reset_index()

    # Gráfico 3: Promedio por mes
    fig.add_trace(
        go.Bar(
            x=monthly_avg['month'],
            y=monthly_avg['y'],
            name='Promedio Mensual',
            marker_color='#2ca02c'
        ),
        row=2, col=1
    )

    # Preparar datos para análisis semanal
    df_weekly = df.copy()
    df_weekly['weekday'] = df_weekly['ds'].dt.day_name()
    weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekly_avg = df_weekly.groupby('weekday')['y'].mean().reindex(weekday_order).reset_index()

    # Gráfico 4: Promedio por día de la semana
    fig.add_trace(
        go.Bar(
            x=weekly_avg['weekday'],
            y=weekly_avg['y'],
            name='Promedio Semanal',
            marker_color='#d62728'
        ),
        row=2, col=2
    )

    # Configurar layout
    fig.update_layout(
        height=800,
        showlegend=False,
        title_text="📊 Análisis Exploratorio de Datos de Ventas",
        title_x=0.5
    )

    # Mostrar gráfico
    fig.show()

    # Estadísticas adicionales
    print("📈 ESTADÍSTICAS DESCRIPTIVAS")
    print("="*50)
    print(f"📊 Media: {df['y'].mean():.1f} unidades/día")
    print(f"📊 Mediana: {df['y'].median():.1f} unidades/día")
    print(f"📊 Mínimo: {df['y'].min():.0f} unidades")
    print(f"📊 Máximo: {df['y'].max():.0f} unidades")
    print(f"📊 Coeficiente de variación: {(df['y'].std()/df['y'].mean()*100):.1f}%")

# Ejecutar análisis exploratorio
try:
    if 'df_prophet' in globals():
        exploratory_analysis(df_prophet)
    else:
        print("⚠️ Primero procesa los datos")
except:
    print("⚠️ Asegúrate de haber procesado los datos correctamente")

📈 ESTADÍSTICAS DESCRIPTIVAS
📊 Media: 154.1 unidades/día
📊 Mediana: 131.0 unidades/día
📊 Mínimo: 69 unidades
📊 Máximo: 743 unidades
📊 Coeficiente de variación: 54.7%


In [ ]:
# CELDA 6: Configuración del modelo Prophet
def create_model_config_widget():
    # Widgets de configuración
    yearly_season = widgets.Checkbox(value=True, description='Estacionalidad Anual')
    weekly_season = widgets.Checkbox(value=True, description='Estacionalidad Semanal')
    daily_season = widgets.Checkbox(value=False, description='Estacionalidad Diaria')

    season_mode = widgets.Dropdown(
        options=['additive', 'multiplicative'],
        value='additive',
        description='Modo Estacional:'
    )

    changepoint_scale = widgets.FloatSlider(
        value=0.05,
        min=0.001,
        max=0.5,
        step=0.001,
        description='Changepoint Scale:'
    )

    seasonality_scale = widgets.FloatSlider(
        value=10.0,
        min=0.01,
        max=50.0,
        step=0.1,
        description='Seasonality Scale:'
    )

    confidence_interval = widgets.FloatSlider(
        value=0.8,
        min=0.5,
        max=0.95,
        step=0.05,
        description='Intervalo Confianza:'
    )

    # Botón para entrenar modelo
    train_button = widgets.Button(
        description='🚀 Entrenar Modelo',
        button_style='success',
        layout=widgets.Layout(width='200px', height='40px')
    )

    output_widget = widgets.Output()

    def on_train_click(b):
        with output_widget:
            output_widget.clear_output()

            try:
                print("🔧 ENTRENANDO MODELO PROPHET")
                print("="*50)

                # Crear modelo Prophet
                model = Prophet(
                    yearly_seasonality=yearly_season.value,
                    weekly_seasonality=weekly_season.value,
                    daily_seasonality=daily_season.value,
                    seasonality_mode=season_mode.value,
                    changepoint_prior_scale=changepoint_scale.value,
                    seasonality_prior_scale=seasonality_scale.value,
                    interval_width=confidence_interval.value
                )

                print("🚀 Entrenando modelo...")
                model.fit(df_prophet)

                # Guardar modelo globalmente
                global prophet_model
                prophet_model = model

                print("✅ Modelo entrenado correctamente!")
                print("🔄 Continúa con la celda 7")

            except Exception as e:
                print(f"❌ Error: {str(e)}")

    train_button.on_click(on_train_click)

    return widgets.VBox([
        widgets.HTML("<h3>⚙️ Configuración del Modelo Prophet</h3>"),
        widgets.HBox([
            widgets.VBox([yearly_season, weekly_season, daily_season]),
            widgets.VBox([season_mode, changepoint_scale]),
            widgets.VBox([seasonality_scale, confidence_interval])
        ]),
        train_button,
        output_widget
    ])

# Mostrar widget de configuración
try:
    if 'df_prophet' in globals():
        config_widget = create_model_config_widget()
        display(config_widget)
    else:
        print("⚠️ Primero procesa los datos")
except:
    print("⚠️ Asegúrate de haber procesado los datos")

In [ ]:
# CELDA 7: Generación de pronósticos
def create_forecast_widget():
    # Widget para seleccionar períodos
    periods_widget = widgets.IntSlider(
        value=120,
        min=30,
        max=365,
        step=30,
        description='Días a pronosticar:'
    )

    # Botón para generar pronóstico
    forecast_button = widgets.Button(
        description='🔮 Generar Pronóstico',
        button_style='info',
        layout=widgets.Layout(width='200px', height='40px')
    )

    output_widget = widgets.Output()

    def on_forecast_click(b):
        with output_widget:
            output_widget.clear_output()

            try:
                print("🔮 GENERANDO PRONÓSTICOS")
                print("="*50)

                # Crear DataFrame futuro
                future = prophet_model.make_future_dataframe(periods=periods_widget.value)

                print(f"📅 Período histórico: {future['ds'].min()} a {df_prophet['ds'].max()}")
                print(f"📅 Período pronóstico: {df_prophet['ds'].max() + timedelta(days=1)} a {future['ds'].max()}")
                print(f"📊 Días a pronosticar: {periods_widget.value}")

                # Generar pronósticos
                print("\n🚀 Calculando pronósticos...")
                forecast = prophet_model.predict(future)

                # Separar datos históricos vs pronósticos
                forecast_data = forecast[forecast['ds'] > df_prophet['ds'].max()].copy()

                print("✅ Pronósticos generados!")

                # Estadísticas del pronóstico
                print(f"\n📈 RESUMEN DEL PRONÓSTICO")
                print("="*50)
                print(f"📊 Promedio pronosticado: {forecast_data['yhat'].mean():.1f} unidades/día")
                print(f"📊 Total pronóstico: {forecast_data['yhat'].sum():.0f} unidades")

                # Comparación con histórico
                historical_avg = df_prophet['y'].mean()
                forecast_avg = forecast_data['yhat'].mean()
                change_pct = ((forecast_avg - historical_avg) / historical_avg) * 100

                print(f"📊 Promedio histórico: {historical_avg:.1f} unidades/día")
                print(f"📊 Cambio esperado: {change_pct:+.1f}%")

                # Guardar resultados globalmente
                global forecast_results, forecast_future
                forecast_results = forecast
                forecast_future = forecast_data

                print("\n🎉 ¡Continúa con la celda 8!")

            except Exception as e:
                print(f"❌ Error: {str(e)}")

    forecast_button.on_click(on_forecast_click)

    return widgets.VBox([
        widgets.HTML("<h3>🔮 Generación de Pronósticos</h3>"),
        periods_widget,
        forecast_button,
        output_widget
    ])

# Mostrar widget de pronósticos
try:
    if 'prophet_model' in globals():
        forecast_widget = create_forecast_widget()
        display(forecast_widget)
    else:
        print("⚠️ Primero entrena el modelo")
except:
    print("⚠️ Asegúrate de haber entrenado el modelo")

In [ ]:
# CELDA 8: Visualización de resultados
def create_forecast_visualizations():
    print("📊 CREANDO VISUALIZACIONES")
    print("="*50)

    # 1. Gráfico principal con Plotly
    fig_main = go.Figure()

    # Datos históricos
    fig_main.add_trace(go.Scatter(
        x=df_prophet['ds'],
        y=df_prophet['y'],
        mode='lines',
        name='Datos Históricos',
        line=dict(color='#1f77b4', width=2)
    ))

    # Pronóstico
    fig_main.add_trace(go.Scatter(
        x=forecast_future['ds'],
        y=forecast_future['yhat'],
        mode='lines',
        name='Pronóstico',
        line=dict(color='#ff7f0e', width=2, dash='dash')
    ))

    # Intervalo de confianza
    fig_main.add_trace(go.Scatter(
        x=forecast_future['ds'],
        y=forecast_future['yhat_upper'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ))

    fig_main.add_trace(go.Scatter(
        x=forecast_future['ds'],
        y=forecast_future['yhat_lower'],
        mode='lines',
        fill='tonexty',
        fillcolor='rgba(255, 127, 14, 0.2)',
        line=dict(width=0),
        name='Intervalo de Confianza',
        hoverinfo='skip'
    ))

    # Configurar layout
    fig_main.update_layout(
        title='📈 Pronóstico de Ventas - Próximos 4 Meses',
        xaxis_title='Fecha',
        yaxis_title='Unidades Vendidas',
        height=600,
        hovermode='x unified',
        showlegend=True
    )

    fig_main.show()

    # 2. Gráfico de componentes
    print("\n📊 Generando gráfico de componentes...")
    fig_components = plot_components_plotly(prophet_model, forecast_results)
    fig_components.show()

    # 3. Métricas de precisión
    print("\n📊 MÉTRICAS DE PRECISIÓN DEL MODELO")
    print("="*50)

    # Obtener predicciones para datos históricos
    historical_forecast = forecast_results[forecast_results['ds'] <= df_prophet['ds'].max()].copy()
    merged_data = pd.merge(df_prophet, historical_forecast[['ds', 'yhat']], on='ds')

    # Calcular métricas
    mae = np.mean(np.abs(merged_data['y'] - merged_data['yhat']))
    mape = np.mean(np.abs((merged_data['y'] - merged_data['yhat']) / merged_data['y'])) * 100
    rmse = np.sqrt(np.mean((merged_data['y'] - merged_data['yhat'])**2))

    print(f"📊 MAE (Error Absoluto Medio): {mae:.2f} unidades")
    print(f"📊 MAPE (Error Porcentual Medio): {mape:.2f}%")
    print(f"📊 RMSE (Raíz del Error Cuadrático Medio): {rmse:.2f} unidades")

    # 4. Análisis mensual del pronóstico
    print(f"\n📅 ANÁLISIS MENSUAL DEL PRONÓSTICO")
    print("="*50)

    forecast_monthly = forecast_future.copy()
    forecast_monthly['month_year'] = forecast_monthly['ds'].dt.to_period('M')
    monthly_summary = forecast_monthly.groupby('month_year').agg({
        'yhat': ['mean', 'sum'],
        'yhat_lower': 'mean',
        'yhat_upper': 'mean'
    }).round(1)

    monthly_summary.columns = ['Promedio_Diario', 'Total_Mensual', 'Limite_Inferior', 'Limite_Superior']
    print(monthly_summary)

    # Guardar resumen mensual globalmente
    global monthly_analysis
    monthly_analysis = monthly_summary

    return fig_main, fig_components

# Ejecutar visualizaciones
try:
    if all(var in globals() for var in ['prophet_model', 'forecast_results', 'forecast_future', 'df_prophet']):
        main_chart, components_chart = create_forecast_visualizations()
        print("\n🎉 ¡Visualizaciones creadas! Continúa con celda 9")
    else:
        print("⚠️ Completa todos los pasos anteriores")
except Exception as e:
    print(f"❌ Error: {str(e)}")

📊 CREANDO VISUALIZACIONES



📊 Generando gráfico de componentes...



📊 MÉTRICAS DE PRECISIÓN DEL MODELO
📊 MAE (Error Absoluto Medio): 26.15 unidades
📊 MAPE (Error Porcentual Medio): 17.85%
📊 RMSE (Raíz del Error Cuadrático Medio): 37.18 unidades

📅 ANÁLISIS MENSUAL DEL PRONÓSTICO
            Promedio_Diario  Total_Mensual  Limite_Inferior  Limite_Superior
month_year                                                                  
2025-01               170.8         5296.3             98.3            242.8
2025-02               164.1         4596.0             92.3            236.0
2025-03               163.3         5062.1             91.2            236.3
2025-04               167.1         5013.7             95.3            239.1

🎉 ¡Visualizaciones creadas! Continúa con celda 9


In [ ]:
# CELDA 9: Exportar resultados
def export_results():
    print("💾 EXPORTANDO RESULTADOS")
    print("="*50)

    model_name = "prophet_forecast"

    try:
        # 1. Exportar pronósticos detallados a CSV
        forecast_export = forecast_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
        forecast_export.columns = ['Fecha', 'Pronostico', 'Limite_Inferior', 'Limite_Superior']
        forecast_export['Fecha'] = forecast_export['Fecha'].dt.strftime('%Y-%m-%d')

        filename_detailed = f"{model_name}_pronostico_detallado.csv"
        forecast_export.to_csv(filename_detailed, index=False)
        files.download(filename_detailed)
        print(f"✅ Exportado: {filename_detailed}")

        # 2. Exportar resumen mensual a CSV
        filename_monthly = f"{model_name}_resumen_mensual.csv"
        monthly_analysis.to_csv(filename_monthly)
        files.download(filename_monthly)
        print(f"✅ Exportado: {filename_monthly}")

        # 3. Crear y exportar gráfico de pronóstico como HTML
        fig_export = go.Figure()

        # Datos históricos
        fig_export.add_trace(go.Scatter(
            x=df_prophet['ds'],
            y=df_prophet['y'],
            mode='lines',
            name='Datos Históricos',
            line=dict(color='#1f77b4', width=2)
        ))

        # Pronóstico
        fig_export.add_trace(go.Scatter(
            x=forecast_future['ds'],
            y=forecast_future['yhat'],
            mode='lines',
            name='Pronóstico',
            line=dict(color='#ff7f0e', width=2, dash='dash')
        ))

        # Intervalo de confianza
        fig_export.add_trace(go.Scatter(
            x=forecast_future['ds'],
            y=forecast_future['yhat_upper'],
            mode='lines',
            line=dict(width=0),
            showlegend=False,
            hoverinfo='skip'
        ))

        fig_export.add_trace(go.Scatter(
            x=forecast_future['ds'],
            y=forecast_future['yhat_lower'],
            mode='lines',
            fill='tonexty',
            fillcolor='rgba(255, 127, 14, 0.2)',
            line=dict(width=0),
            name='Intervalo de Confianza',
            hoverinfo='skip'
        ))

        fig_export.update_layout(
            title='📈 Pronóstico de Ventas - Próximos 4 Meses',
            xaxis_title='Fecha',
            yaxis_title='Unidades Vendidas',
            height=600,
            hovermode='x unified'
        )

        # Exportar gráfico
        filename_chart = f"{model_name}_grafico.html"
        fig_export.write_html(filename_chart)
        files.download(filename_chart)
        print(f"✅ Exportado: {filename_chart}")

        # 4. Crear reporte de resumen
        total_forecast = forecast_future['yhat'].sum()
        avg_daily = forecast_future['yhat'].mean()
        historical_avg = df_prophet['y'].mean()
        change_pct = ((avg_daily - historical_avg) / historical_avg) * 100

        report_text = f"""
REPORTE DE PRONÓSTICOS DE VENTAS
================================

Modelo: Prophet
Fecha de generación: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Período de pronóstico: {forecast_future['ds'].min().strftime('%Y-%m-%d')} a {forecast_future['ds'].max().strftime('%Y-%m-%d')}

RESUMEN EJECUTIVO:
- Total pronosticado (4 meses): {total_forecast:.0f} unidades
- Promedio diario pronosticado: {avg_daily:.1f} unidades
- Promedio histórico: {historical_avg:.1f} unidades
- Cambio esperado: {change_pct:+.1f}%

ESTADÍSTICAS DEL PRONÓSTICO:
- Mínimo: {forecast_future['yhat'].min():.0f} unidades
- Máximo: {forecast_future['yhat'].max():.0f} unidades
- Desviación estándar: {forecast_future['yhat'].std():.1f} unidades

RESUMEN MENSUAL:
{monthly_analysis.to_string()}
        """

        # Exportar reporte
        filename_report = f"{model_name}_reporte.txt"
        with open(filename_report, 'w', encoding='utf-8') as f:
            f.write(report_text)
        files.download(filename_report)
        print(f"✅ Exportado: {filename_report}")

        print(f"\n🎉 ¡Todos los archivos exportados!")
        print("📁 Archivos generados:")
        print(f"   • {filename_detailed}")
        print(f"   • {filename_monthly}")
        print(f"   • {filename_chart}")
        print(f"   • {filename_report}")

    except Exception as e:
        print(f"❌ Error al exportar: {str(e)}")

# Botón para ejecutar exportación
export_button = widgets.Button(
    description='💾 Exportar Todo',
    button_style='warning',
    layout=widgets.Layout(width='200px', height='40px')
)

output_export = widgets.Output()

def on_export_click(b):
    with output_export:
        output_export.clear_output()
        export_results()

export_button.on_click(on_export_click)

# Mostrar interfaz de exportación
try:
    if all(var in globals() for var in ['forecast_future', 'monthly_analysis', 'df_prophet']):
        display(widgets.VBox([
            widgets.HTML("<h3>💾 Exportación de Resultados</h3>"),
            export_button,
            output_export
        ]))
    else:
        print("⚠️ Completa el análisis antes de exportar")
except:
    print("⚠️ Completa todos los pasos anteriores")